## MNIST challenge - ConvNet model 1


In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 

print(tf.__version__)

2.4.1


In [4]:
from tensorflow.python.framework import ops

def reset_graph(seed=42):
    ops.reset_defult_graph()
    tf.random.set_seed(seed)
    np.random.seed(seed)

build the CNN architecture

In [2]:
#reset_graph()

with tf.name_scope("input"):
    X = tf.placeholder(tf.float32, shape=[None, 28*28], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, 28, 28, 1])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=32, kernel_size=3, strides=1, padding="SAME", activation=tf.nn.relu, name="conv1")

conv2 = tf.layers.conv2d(X_reshaped, filters=64, kernel_size=3, strides=1, padding="SAME", activation=tf.nn.relu, name="conv1")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], stride=[1,2,2,1], padding="VAlID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_flat*7*7])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, 64, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, 10, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.


ValueError: Variable conv1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "f:\JetBrain Project Files\Pycharm\MNIST-challenge\.virtualenv\lib\site-packages\tensorflow_core\python\framework\ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "f:\JetBrain Project Files\Pycharm\MNIST-challenge\.virtualenv\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "f:\JetBrain Project Files\Pycharm\MNIST-challenge\.virtualenv\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "f:\JetBrain Project Files\Pycharm\MNIST-challenge\.virtualenv\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "f:\JetBrain Project Files\Pycharm\MNIST-challenge\.virtualenv\lib\site-packages